In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
df = pd.read_csv('./data2/AXISBANK.csv')
# Calculate 'H-L' column
df['H-L'] = df['High'] - df['Low']

# Calculate 'O-C' column
df['O-C'] = df['Close'] - df['Open']

# Calculate '7 DAYS MA' column
df['7 DAYS MA'] = df['Open'].rolling(window=7).mean()

# Calculate '14 DAYS MA' column
df['14 DAYS MA'] = df['Open'].rolling(window=14).mean()

# Calculate '21 DAYS MA' column
df['21 DAYS MA'] = df['Open'].rolling(window=21).mean()

# Calculate '7 DAYS STD DEV' column
df['7 DAYS STD DEV'] = df['Open'].rolling(window=7).std()

# df['Date'] = pd.to_datetime(df['Date']) 

In [ ]:
df_new = df.iloc[20:5036,[3,4,5,6,7,8,9,10,11,15,16,17,18,19,20]]
y_val = df.iloc[21:5037,4].dropna()
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_new)

In [ ]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_dim=df_scaled.shape[1]),
    layers.Dropout(0.2),  # Adding dropout for regularization
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='linear')  # Linear activation for regression
])

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(df_scaled, y_val, epochs=100, batch_size=32, verbose=1)


In [ ]:
def predict_open(df,trueY):
    model = keras.models.load_model("NN_model.h5")
    scaled_input = scaler.transform(df.iloc[-1:, [3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16, 17, 18, 19, 20]])
    model.fit(scaled_input, np.array([trueY]), epochs=100, batch_size=32, verbose=1)
    predicted_open = model.predict(scaled_input)
    model.save("NN_model.h5")
    return predicted_open


In [ ]:
predicted={}
for i in range(5037,5305):
    row = df.iloc[:i]
    trueY = df.iloc[i+1,4]
    # row_df = pd.DataFrame([row])
    predicted[df.iloc[i,0]]=[trueY,predict_open(row,trueY)]


In [ ]:
xvalues = list(map(lambda x:x.date(),predicted.keys()))
trueys = list(map(lambda x:x[0],predicted.values()))
predictedy = list(map(lambda x:x[1][0][0],predicted.values()))

In [ ]:
import plotly.graph_objects as go
trace1 = go.Scatter(x=xvalues, y=trueys, mode='lines', name='True Open')
trace2 = go.Scatter(x=xvalues, y=predictedy, mode='lines', name='Predicted Open')
layout = go.Layout(title='Two Line Graphs', xaxis=dict(title='X-axis'), yaxis=dict(title='Y-axis'))
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()
fig.write_image("NN predicted price.png")
